In [ ]:

import cv2



In [ ]:
# face_cap=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
# video_cap=cv2.VideoCapture(0) 
#     ret,video_data=video_cap.read()
#     col=cv2.cvtColor(video_data,cv2.COLOR_BGR2GRAY)
#     faces=face_cap.detectMultiScale(
#         col,
#         scaleFactor=1.1,
#         minNeighbors=5,
#         minSize=(30,30),
#         flags=cv2.CASCADE_SCALE_IMAGE
#     )
#     for(x,y,w,h) in faces:
#         cv2.rectangle(video_data,(x,y),(x+w,y+h),(0,255,0),2)
#     cv2.imshow("video_live",video_data)
#     if cv2.waitKey(10)==ord("a"):
#         break
# video_cap.release()

In [ ]:
# %pip3 install --upgrade pip
# %pip3 install --upgrade tensorflow


In [ ]:
# %pip install tensorflow

In [ ]:
import os
import sys
import json
import tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Activation, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

In [ ]:
num_classes=7
img_rows,img_cols=48,48
batch_size=32

In [ ]:
train_data='images/train'
validation_data='images/validation'

In [ ]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen=ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator=train_datagen.flow_from_directory(
    train_data,
    color_mode="grayscale",
    target_size=(img_rows,img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)
validation_generator=validation_datagen.flow_from_directory(
    validation_data,
    color_mode="grayscale",
    target_size=(img_rows,img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)




In [ ]:
model=Sequential()

In [ ]:
# Block-1
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Block-2
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Block-3
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Block-4
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Block-5
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#Block-6
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#Block-7
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [ ]:
print(model.summary())

In [ ]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [ ]:
checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)
callbacks = [earlystop,checkpoint,reduce_lr]

In [ ]:
model.compile(loss='categorical_crossentropy',
 optimizer = Adam(lr=0.001),
 metrics=['accuracy'])
nb_train_samples = 28821
nb_validation_samples = 7066
epochs=25
history=model.fit(
 train_generator,
 steps_per_epoch=nb_train_samples//batch_size,
 epochs=epochs,
 callbacks=callbacks,
 validation_data=validation_generator,
 validation_steps=nb_validation_samples//batch_size)

In [80]:
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

In [81]:
face_classifier=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
classifier=load_model('EmotionDetectionModel.h5')


In [82]:
class_labels=['Angry','disgust','fear','happy','neutral','sad','surprise']
cap=cv2.VideoCapture(0)

In [83]:
while True:
    ret,frame=cap.read()
    labels=[]
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)

    for(x,y,w,h) in faces:
       cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
       roi_gray=gray[y:y+h,x:x+w]
       roi_gray=cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

       if np.sum([roi_gray])!=0:
           roi=roi_gray.astype('float')/255.0
           roi=img_to_array(roi)
           roi=np.expand_dims(roi,axis=0)


           preds=classifier.predict(roi)[0]
           label=class_labels[preds.argmax()]
           label_position=(x,y)
           cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
       else:
           cv2.putText(frame,'No Face Found',(20,20),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)

    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
       break    
cap.release()    
cv2.destroyAllWindows()





1/1 [==============================] - 0s 30ms/step


In [ ]:
if np.sum([roi_gray])!=0:
   roi=roi_gray.astype('float')/255.0
   roi=img_to_array(roi)
   roi=np.expand_dims(roi,axis=0)
   preds=classifier.predict(roi)[0]
   label=class_labels[preds.argmax()]
   label_position=(x,y)
   cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
else:
   cv2.putText(frame,'No Face Found',(20,20),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
cv2.imshow('Emotion Detector',frame)
if cv2.waitKey(1) & 0xFF == ord('q'): 
   break